## ***Machine Learning Products (MLOps)***

#### En este cuaderno jupiter se encuentra todo el proceso del Machine Learning, desde la conexión a la base de datos hasta el sistema de recomendación


#### 1. **Importamos librerias necesarias**

In [97]:
# Importamos librerias 
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import os 
import env

#### 2. **Conectamos con la base de datos**

In [ ]:
# Si existen cambios en la credenciales, refrescamos el env
import importlib
import env

importlib.reload(env)

<module 'env' from 'd:\\Personal\\Documentos\\Courses\\Henry\\Henry Students\\Labs\\Cohorte 25 ft\\PG_FT_25\\Proyecto_final_Henry\\Machine_Learning\\env.py'>

In [93]:
# credenciales del AWS

HOST_DB = env.DB_HOST
USER_DB = env.DB_USER
PASS_DB = env.DB_PASS
NAME_DB = env.DB_NAME

In [103]:
# Conectamos con la base de datos 

def BD_connection(host, user_db, user_pass, name_db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user_db,
            passwd=user_pass,
            database=name_db,
            port=3306,
            connection_timeout=300
        )
        print("MYSQL DATABASE connection succesful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [96]:
connection = BD_connection(HOST_DB, USER_DB, PASS_DB, NAME_DB)

MYSQL DATABASE connection succesful


----------------------------------------------------